### 텍스트 데이터 다루기

> 텍스트에서 중요한 거 "오더" : 텍스트의 오더를 기준으로 텍스트를 숫자로 바꿔주는 알고리즘을 알아야 함

> 여기서 필요한건 원핫인코딩이 아니라 embedding

> 한국어는 오더가 안 중요하기 때문에(도치해도 같은 의미) 별도처리


In [39]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import mglearn
import koreanize_matplotlib

from preamble import *
from sklearn.datasets import load_files
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import GridSearchCV

### 데이터 불러오기

In [17]:
reviews_train = load_files("data/aclImdb/train/")

In [18]:
text_train, y_train = reviews_train.data, reviews_train.target

In [19]:
print(text_train[6])

b'Gloomy Sunday - Ein Lied von Liebe und Tod directed by Rolf Sch\xc3\xbcbel in 1999 is a romantic, absorbing, beautiful, and heartbreaking movie. It started like Jules and Jim; it ended as one of Agatha Christie\'s books, and in between it said something about love, friendship, devotion, jealousy, war, Holocaust, dignity, and betrayal, and it did better than The Black Book which is much more popular. It is not perfect, and it made me, a cynic, wonder in the end on the complexity of the relationships and sensational revelations, and who is who to whom but the movie simply overwhelmed me. Perfect or not, it is unforgettable. All four actors as the parts of the tragic not even a triangle but a rectangle were terrific. I do believe that three men could fell deeply for one girl as beautiful and dignified as Ilona in a star-making performance by young Hungarian actress Erica Marozs\xc3\xa1n and who would not? The titular song is haunting, sad, and beautiful, and no doubt deserves the movie 

In [21]:
#리스트컴프리
text_train = [doc.replace(b"<br />", b" ") for doc in text_train]

### 텍스트 임베딩

*  텍스트 그대로 쓸 수 없으니까 숫자로 바꾼다 

*  대표적 : BOW  (빈 도 ) "CountVectorizer"

     * P) 순서 같은 거 무시하고 횟수당으로 계산때리므로 계산이 쉬우나
     
     * N) 한번만 나와도 다 걸림 -> 전체를 다 보므로 데이터 편향이 높음-> 
     
      -> 한 도메인에는 꽉 들어차게 잘 맞는데 아닌거에는 대처를 못하는거지 => "tf-idf"등장

*  tf-idf (역문서 빈 도 )
      
      * 배제하는 단어가 생긴다는 점- 모든 문서에 다 등장하는 건 중요도를 낮추고 한 문서에 많이 나온건 가중치를 부여함 

      * 보통명사들을 중요하지 않게 처리하고 좀 특이한 단어에 가중치를 부여하는 점이 있음 => 섞어써.. "Ngram"

* Ngram
      
      * bow가 토큰 하나의 횟수만 고려했던 것 -> Ngram은 옆에 있는 두 세개 토큰을 함께 "문맥"을 고려함

      * 확률로 붙을 단어들이 예측 가능해짐

      * 연관검색어

In [23]:
#CountVectorizer는 토큰화하는거랑 어휘사전화 하는거까지 다 포함하는 처리과정 
vect = CountVectorizer().fit(text_train)
X_train=vect.transform(text_train)

In [24]:
#어휘 사전으로 만들어낸걸 반납함
vect.vocabulary_

{'full': 43186,
 'of': 78010,
 'then': 110005,
 'unknown': 115586,
 'actors': 2987,
 'tsf': 113230,
 'is': 56764,
 'great': 47036,
 'big': 12418,
 'cuddly': 26587,
 'romp': 93359,
 'film': 40335,
 'the': 109916,
 'idea': 53513,
 'bunch': 16409,
 'bored': 14422,
 'teenagers': 109094,
 'ripping': 92688,
 'off': 78019,
 'local': 64984,
 'sink': 100248,
 'factory': 38685,
 'odd': 77919,
 'enough': 36245,
 'but': 16792,
 'add': 3080,
 'in': 54396,
 'black': 12926,
 'humour': 52921,
 'that': 109890,
 'forsyth': 42072,
 'co': 22164,
 'are': 7054,
 'so': 101860,
 'good': 46249,
 'at': 8095,
 'and': 5618,
 'your': 123231,
 'for': 41777,
 'real': 89498,
 'treat': 112525,
 'comatose': 22824,
 'van': 117102,
 'driver': 33291,
 'by': 16930,
 'itself': 57001,
 'worth': 122032,
 'seeing': 97260,
 'canal': 17593,
 'side': 99720,
 'chase': 19874,
 'just': 58901,
 'too': 111517,
 'to': 111180,
 'be': 10846,
 'anything': 6360,
 'funny': 43319,
 'anyone': 6356,
 'who': 120719,
 'lived': 64804,
 'glasgow':

In [26]:
#얘는 어휘 사전 하기 전에 그냥 싹다 토큰화한 결과를 출력함
features_names = vect.get_feature_names_out()
features_names

array(['00', '000', '0000', ..., 'книги', '色戒', 'ｒｏｃｋ'], dtype=object)

### Baseline

> 아무것도 안했을때 베이스라인 보려고

In [27]:
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression

scores = cross_val_score(LogisticRegression(), X_train, y_train, n_jobs=-1)

In [28]:
print(np.mean(scores))

0.67096


In [38]:
# from sklearn.model_selection import GridSearchCV
# param_grid = {"C": [0.001, 0.01, 0.1, 1, 10]}
# grid = GridSearchCV(LogisticRegression(max_iter=5000), param_grid=param_grid, n_jobs=-1)
# grid.fit(X_train, y_train)
# print(grid.best_score_)
# 오래걸려서 주석처리함 ^^

### 불용어 처리

In [ ]:
#CountVectorizer 하는데 단어가 나타나는 최소 비율은 5개. 5개보다 적게 등장하면 무시해~
vect = CountVectorizer(min_df=5, stop_words="english").fit(text_train)
X_train = vect.transform(text_train)
features_names = vect.get_feature_names_out()

In [ ]:
param_grid = {"C": [0.001, 0.01, 0.1, 1, 10]}
grid = GridSearchCV(LogisticRegression(max_iter=5000), param_grid=param_grid, n_jobs=-1)
grid.fit(X_train, y_train)
print(grid.best_score_)

### N-그램

* 기준이 되어줄만한 어휘사전 <코퍼스>가 있어야 함

* 주로 2개를 많이 봄

* 한국어는 띄어쓰기없이 쓰는 경우 때문에 bow가 안먹히잖아 그래서 한국어 처리는 안에 다르게 조사까지 다 뜯어주든지 하는걸 추가해야됨

In [32]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.pipeline import make_pipeline

In [33]:
pipe = make_pipeline(TfidfVectorizer(min_df=5), LogisticRegression(max_iter=100))


In [36]:
# TfidfVectorizer 객체 생성, 2-그램 포함
# vectorizer = TfidfVectorizer(ngram_range=(2, 2)).fit(text_train)
# 다른 이름이 있는게 아니고 tfidf 할때 ngram에 range를 정해줌  ! 

param_grid = { "logisticregression__C" : [10, 100],
               "tfidfvectorizer__ngram_range": [(1,1), (1,2), (1,3)]}
                                             #단일단어, 2단어, 3단어 n-그램으로 사용한다


In [37]:
grid = GridSearchCV(pipe, param_grid, n_jobs=-1)
grid.fit(text_train, y_train)
grid.best_score_

c:\Users\user\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


0.7110133333333334